### Leslie Matrix

&nbsp;

The Leslie matrix is a square matrix to model and analyze the dynamics of age-structured populations. It is a projection matrix to study the growth and stability of populations over time.

In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
os.chdir('C:/Users/tm/Downloads/utas/thesis/chapter1/tasman')

In [2]:
#number of columns and rows in leslie matrix
maxage=5

### cleanse

In [3]:
dataset=[]
for i in os.listdir('./Data collation for Forestier Peninsula'):
    if 'xlsx' not in i:
        filename=os.listdir(f'./Data collation for Forestier Peninsula/{i}')[0]
        spreadsheet=pd.ExcelFile(f'./Data collation for Forestier Peninsula/{i}/{filename}')        
        dataset.append(spreadsheet.parse(spreadsheet.sheet_names[0]))

C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\tm\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default sty

In [4]:
df=pd.concat([i for i in dataset])

In [5]:
#sort by date
grande=df[['INDIVIDUAL','OBSERVATION_DATE','BIRTH_DATE','NUMBER_ACTIVE_TEATS','GENDER']].sort_values(
    ['INDIVIDUAL','OBSERVATION_DATE'])

#female parent only
grande=grande[grande['GENDER']=='Female']

#female offspring only
grande['NUMBER_ACTIVE_TEATS']/=2
grande.reset_index(inplace=True,drop=True)

#datetimeindex
grande['OBSERVATION_DATE']=pd.to_datetime(grande['OBSERVATION_DATE'],format='mixed')
grande['BIRTH_DATE']=pd.to_datetime(grande['BIRTH_DATE'])

In [6]:
#eliminate wrong birth date
grande=grande[grande['OBSERVATION_DATE']>grande['BIRTH_DATE']]

#eliminate null id
grande=grande.loc[grande['INDIVIDUAL'].dropna().index]

In [7]:
#work on disease natural growth
grande['year']=grande['OBSERVATION_DATE'].dt.year
grande=grande[grande['year']<2015]

### estimate transition rate

In [8]:
#compute age
grande['age']=(grande['OBSERVATION_DATE']-grande['BIRTH_DATE']).apply(lambda x:x.days//365)

In [9]:
#for each year,each devil only counts once
grande['year']=grande['OBSERVATION_DATE'].dt.year
agestructure=grande.loc[grande[['year','INDIVIDUAL']].drop_duplicates().index]

In [10]:
#count
agesummary=agestructure.groupby('age').count()[['INDIVIDUAL']]
agesummary.reset_index(inplace=True)

In [11]:
#remove devil older than 10 yr which should be error
agesummary=agesummary[agesummary['age']<=10]

#move anything older than maxage yr into yr maxage category
lastrow=agesummary['INDIVIDUAL'][agesummary['age']>=maxage].sum()
agesummary['INDIVIDUAL'][agesummary['age']==maxage]=lastrow
agesummary=agesummary.iloc[:maxage+1]

In [12]:
#estimate relationship between age and frequency
#plus one to avoid log(0)
agesummary['logage']=np.log(agesummary['age']+1)

m=sm.OLS(agesummary['INDIVIDUAL'],sm.add_constant(agesummary['logage'])).fit()

In [13]:
#use fitted data to compute survivalship
agesummary['smoothed data']=m.predict()
agesummary['survivalship']=agesummary['smoothed data']/agesummary['smoothed data'].iloc[0]

### estimate fecundity

In [14]:
#for each year,each devil only counts once
fecundity=grande.groupby(['year','INDIVIDUAL']).max()[['NUMBER_ACTIVE_TEATS','age','GENDER']]
fecundity.reset_index(inplace=True)

In [15]:
#remove na female
female=fecundity[fecundity['GENDER']=='Female'].copy()
female=female.loc[female['NUMBER_ACTIVE_TEATS'].dropna().index]

#set na male to zero
male=fecundity[fecundity['GENDER']=='Male'].copy()
fecundity=pd.concat([female,male])
fecundity['NUMBER_ACTIVE_TEATS']=fecundity['NUMBER_ACTIVE_TEATS'].fillna(0)

In [16]:
#comput mean fecundity
reproduction=fecundity[['NUMBER_ACTIVE_TEATS','age']].groupby('age').mean()
reproduction.reset_index(inplace=True)

In [17]:
#remove devil older than 10 yr which should be error
reproduction=reproduction[reproduction['age']<=10]

#move anything older than maxage yr into yr maxage category
lastrow=reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']>=maxage].mean()
reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']==maxage]=lastrow
reproduction=reproduction.iloc[:maxage+1]

C:\Users\tm\AppData\Local\Temp\ipykernel_34124\3778900559.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reproduction['NUMBER_ACTIVE_TEATS'][reproduction['age']==maxage]=lastrow


### compute leslie matrix

In [18]:
#generate leslie matrix
lesliematrix=np.zeros((maxage+1,maxage+1))
lesliematrix[0]=reproduction['NUMBER_ACTIVE_TEATS'].tolist()
for i in range(1,maxage+1):
    lesliematrix[i][i-1]=agesummary['survivalship'].iloc[i]

In [19]:
#show no disease matrix
for i in lesliematrix:
    print(i)

[0.07407407 0.212      1.59803922 1.71052632 1.55       0.64772727]
[0.73785791 0.         0.         0.         0.         0.        ]
[0.         0.58451461 0.         0.         0.         0.        ]
[0.         0.         0.47571581 0.         0.         0.        ]
[0.         0.         0.         0.39132491 0.         0.        ]
[0.         0.         0.         0.         0.32237252 0.        ]


In [20]:
#dominant eigenvalue still smaller than one
#the population eventually will decline
max(np.abs(np.linalg.eigvals(lesliematrix)))

1.1140958995275123

In [21]:
lesliematrix

array([[0.07407407, 0.212     , 1.59803922, 1.71052632, 1.55      ,
        0.64772727],
       [0.73785791, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.58451461, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.47571581, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.39132491, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.32237252,
        0.        ]])